In [11]:
from torch import cuda, bfloat16
import transformers
from huggingface_hub import notebook_login
notebook_login()

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, you need an access token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,)


model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
)

stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)



INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp14pe6k32
Created a temporary directory at /tmp/tmp14pe6k32
Created a temporary directory at /tmp/tmp14pe6k32
Created a temporary directory at /tmp/tmp14pe6k32
Created a temporary directory at /tmp/tmp14pe6k32
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp14pe6k32/_remote_module_non_scriptable.py
Writing /tmp/tmp14pe6k32/_remote_module_non_scriptable.py
Writing /tmp/tmp14pe6k32/_remote_module_non_scriptable.py
Writing /tmp/tmp14pe6k32/_remote_module_non_scriptable.py
Writing /tmp/tmp14pe6k32/_remote_module_non_scriptable.py


In [8]:
documents = SimpleDirectoryReader('transcripts').load_data()